In [82]:
#把所有的csv文件移动到一起
import os
import shutil

# 设置原始文件夹路径（WSL路径）
#folder_path = '/mnt/e/ChEMBL_Dock_release/chembl_smina_uniprot_in_pdbbind_site_specific_docking/chembl_smina_uniprot_in_pdbbind_site_specific_docking'
#folder_path = '/mnt/e/ChEMBL_Dock_release/chembl_smina_uniprot_in_pdbbind_blind_docking/chembl_smina_uniprot_in_pdbbind_blind_docking'
folder_path = '/mnt/e/ChEMBL_Dock_release/chembl_smina_uniprot_not_in_pdbbind_blind_docking/chembl_smina_uniprot_not_in_pdbbind_blind_docking'

# 设置目标文件夹路径（WSL路径）
#new_folder_path ='/mnt/e/specific_docking'
#new_folder_path = '/mnt/e/blind_docking'
new_folder_path = '/mnt/e/no_blind_docking'

# 创建目标文件夹
os.makedirs(new_folder_path, exist_ok=True)

# 遍历原始文件夹中的所有子文件夹
for root, dirs, files in os.walk(folder_path):
    for folder in dirs:
        folder_fullpath = os.path.join(root, folder)

        # 获取并复制子文件夹中的所有 CSV 文件到目标文件夹中
        for file in os.listdir(folder_fullpath):
            if file.endswith('.csv'):
                shutil.copy2(os.path.join(folder_fullpath, file), new_folder_path)

print('CSV 文件已成功复制到新的文件夹中。')


CSV 文件已成功复制到新的文件夹中。


In [83]:
#读取所有文件的uniprot
import os
import pandas as pd
import shutil

# 设置原始文件夹路径
#folder_path ='/mnt/e/specific_docking'
#folder_path ='/mnt/e/blind_docking'
folder_path = '/mnt/e/no_blind_docking'

# 设置目标文件夹路径
#new_folder_path = '/mnt/e/specific_docking_uniprot'
#new_folder_path = '/mnt/e/blind_docking_uniprot'
new_folder_path = '/mnt/e/no_blind_docking_uniprot'

# 获取文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 创建目标文件夹
os.makedirs(new_folder_path, exist_ok=True)

# 遍历CSV文件
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    # 添加新列，列名为'uniprot'，值为文件名
    df['uniprot'] = csv_file
    
    # 保存修改后的CSV文件到目标文件夹中
    modified_file_path = os.path.join(new_folder_path, csv_file)
    df.to_csv(modified_file_path, index=False)

print('操作完成。')


操作完成。


In [85]:
#把读取结果合并为一个csv
import os
import pandas as pd

# 设置文件夹路径
#folder_path = '/mnt/e/specific_docking_uniprot'
#folder_path = '/mnt/e/blind_docking_uniprot'
folder_path = '/mnt/e/no_blind_docking_uniprot'

#output_file_path = '/mnt/e/chembl/chembl_spec.csv'
#output_file_path = '/mnt/e/chembl/chembl_blind.csv'
output_file_path = '/mnt/e/chembl/chembl_no_blind.csv'

# 获取文件夹中的所有CSV文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 创建一个空的DataFrame来存储合并后的数据
merged_data = pd.DataFrame()

# 遍历CSV文件
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    # 将当前CSV文件的数据添加到合并后的DataFrame中
    merged_data = pd.concat([merged_data, df], ignore_index=True)

# 将合并后的数据保存为CSV文件
merged_data.to_csv(output_file_path, index=False)

print('操作完成。')



操作完成。


In [86]:
#修改uniprot的格式
import pandas as pd

# 读取CSV文件
#df = pd.read_csv('/mnt/e/chembl/chembl_spec.csv')
#df = pd.read_csv('/mnt/e/chembl/chembl_blind.csv')
df = pd.read_csv('/mnt/e/chembl/chembl_no_blind.csv')



# 定义一个函数用于修改每行的值
def modify_value(value):
    return str(value).replace('_info.csv', '')

# 使用apply方法在第十六列上应用函数
df['uniprot'] = df['uniprot'].apply(modify_value)

# 保存修改后的数据为CSV文件
#df.to_csv('/mnt/e/chembl/chembl_spec_end.csv', index=False)
#df.to_csv('/mnt/e/chembl/chembl_blind_end.csv', index=False)
df.to_csv('/mnt/e/chembl/chembl_no_blind_end.csv', index=False)

In [87]:
#对smiles添加convert_to_canonical_smiles
import csv
from rdkit import Chem

def convert_to_canonical_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES")

    canon_smiles = Chem.MolToSmiles(mol, isomericSmiles=False, canonical=True)

    return canon_smiles

# 打开CSV文件
with open('/mnt/e/chembl/chembl_no_blind_end.csv', 'r') as file:
#with open('/mnt/e/chembl/chembl_blind_end.csv', 'r') as file:
#with open('/mnt/e/chembl/chembl_spec_end.csv', 'r') as file:
    # 创建CSV读取器和写入器
    reader = csv.reader(file)
    writer_rows = []

    # 逐行处理数据
    for i, row in enumerate(reader):
        if i == 0:
            # 添加"Canonical SMILES"列标题到第17列
            row.append("Canonical SMILES")
        else:
            try:
                # 将第16列的值转化为Canonical SMILES
                value = row[13]  # 第16列的索引为15
                canonical_smiles = convert_to_canonical_smiles(value)
                # 将Canonical SMILES添加到第17列
                row.append(canonical_smiles)
            except (IndexError, ValueError):
                # 如果无法转换为Canonical SMILES，将空值添加到第17列
                row.append("")

        # 添加当前行到写入器列表
        writer_rows.append(row)

# 更新CSV文件
with open('/mnt/e/chembl/chembl_no_blind_end_end.csv', 'w', newline='') as file:
#with open('/mnt/e/chembl/chembl_blind_end_end.csv', 'w', newline='') as file:
#with open('/mnt/e/chembl/chembl_spec_end_end.csv', 'w', newline='') as file:
    # 创建CSV写入器
    writer = csv.writer(file)
    # 写入数据
    writer.writerows(writer_rows)


In [80]:
#gostar添加Canonical
import csv
from rdkit import Chem

def convert_to_canonical_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Invalid SMILES")

    canon_smiles = Chem.MolToSmiles(mol, isomericSmiles=False, canonical=True)

    return canon_smiles

# 打开CSV文件
with open('/mnt/e/chembl/mysql_all.csv', 'r') as file:
    reader = csv.reader(file)
    writer_rows = []

    # 逐行处理数据
    for i, row in enumerate(reader):
        # 处理每一行的数据
        if i == 0:
            # 添加"Canonical SMILES"列标题到第10列
            row.insert(9, "Canonical SMILES")
        else:
            try:
                # 将第6列的值转化为Canonical SMILES
                value = row[5]  # 第6列的索引为5
                canonical_smiles = convert_to_canonical_smiles(value)
                # 在第10列插入Canonical SMILES
                row.insert(9, canonical_smiles)
            except (IndexError, ValueError):
                # 如果无法转换为Canonical SMILES，将空值添加到第10列
                row.insert(9, "")

        # 添加当前行到写入器列表
        writer_rows.append(row)

# 更新CSV文件
with open('/mnt/e/chembl/mysql_all_end_end.csv', 'w', newline='') as file:
    # 创建CSV写入器
    writer = csv.writer(file)
    # 写入数据
    writer.writerows(writer_rows)



[11:55:03] Explicit valence for atom # 21 N, 4, is greater than permitted
[11:55:04] Explicit valence for atom # 25 N, 5, is greater than permitted
[11:55:04] Explicit valence for atom # 13 N, 4, is greater than permitted
[11:55:05] Explicit valence for atom # 23 N, 4, is greater than permitted
[11:55:06] Explicit valence for atom # 24 N, 4, is greater than permitted
[11:55:06] Explicit valence for atom # 39 N, 4, is greater than permitted
[11:55:06] Explicit valence for atom # 5 N, 4, is greater than permitted
[11:55:06] Explicit valence for atom # 14 N, 4, is greater than permitted
[11:55:08] Explicit valence for atom # 2 N, 4, is greater than permitted
[11:55:08] Explicit valence for atom # 24 N, 4, is greater than permitted
[11:55:08] WARNING: not removing hydrogen atom without neighbors
[11:55:09] Explicit valence for atom # 21 N, 4, is greater than permitted
[11:55:09] Explicit valence for atom # 20 N, 4, is greater than permitted
[11:55:09] Explicit valence for atom # 14 N, 4, i

In [76]:
#对比环节
import csv

def search_csv(csv_file, value_set):
    # 创建输出文件
    output_file = '/mnt/e/chembl/shaixuan2.csv'
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)

        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[1] in value_set:
                    row.append('1')
                else:
                    row.append('0')
                writer.writerow(row)  # 写入结果到输出文件

    print('结果已保存到', output_file)

# 示例用法
value_set = set()  # 创建空集合
csv_file = '/mnt/e/chembl/shaixuan.csv'

# 从CSV文件的第一列构建集合
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        value_set.add(row[0])

# 搜索CSV文件的第二列，并保存匹配结果到CSV文件
search_csv(csv_file, value_set)


结果已保存到 /mnt/e/chembl/shaixuan2.csv
